In [79]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [80]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [81]:
type(df_train)

pandas.core.frame.DataFrame

In [82]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [83]:
df_train.shape

(75000, 4)

In [84]:
df_valid=df_test.copy()

In [85]:
df_train=df_train.loc[df_train['label']<2]

In [86]:
df_train.shape

(25000, 4)

In [87]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [88]:
type(df_train['tokens'])

pandas.core.series.Series

In [89]:
bs=52 # 52 - Jeremey, 20 - default

In [90]:
bptt= 70 #70 - Jeremey, 35 - default

In [91]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [92]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [93]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [94]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [95]:
padlen=1400
padding_idx=1

In [96]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [97]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [98]:
df_train.sort_values(by='n_tok', inplace=True)

In [99]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [100]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [101]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [102]:
df_train.shape

(25000, 5)

In [103]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [104]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [105]:
padlen

1400

In [106]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [107]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [108]:
padlen

1400

In [109]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [110]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [111]:
for xb,yb,xlen in dltrain:
    break

In [112]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
         1, 0, 1, 1]),
 tensor([162, 487, 589, 320, 188, 171, 185, 308, 152,  91, 157, 298, 450, 165,
         118, 344, 191, 167, 241, 167, 186, 404, 198, 152, 264, 140, 166, 527,
         176,  77, 225, 239, 563, 283, 193, 184, 139, 154, 727, 178, 122, 196,
         301, 332,  69, 307, 223, 145, 188,  53, 578, 275]))

## Model Architecture

In [113]:
from torch import nn
import torch
from torch.autograd import Variable

In [114]:
bs,bptt

(52, 70)

In [115]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.2 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [116]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [117]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [118]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [119]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [120]:
import torch

In [121]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [122]:
m(input).size()

torch.Size([1, 64, 5])

In [125]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        
        self.fc1=nn.Linear(len(self.filter_sizes)*self.n_filters+self.n_hidden*4,self.n_hidden)
        self.relu=torch.nn.ReLU()
        self.fc2=nn.Linear(self.n_hidden,100)
        self.fc3=nn.Linear(100,self.n_out)

        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs)) 
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)
        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        
        ## Concatenate
        big_out=torch.cat([cat_cnn,hidden,max_pool],dim=1)
        preds=self.fc3(self.relu(self.fc2(self.relu(self.fc1(big_out)))))

        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))

        
        return preds.view(-1),loss
        
    

In [126]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [127]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [128]:
device="cpu"

## Load pretrained model weights

In [129]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [130]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [131]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [132]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [133]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 31,682,501 trainable parameters


In [134]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc1): Linear(in_features=1900, out_features=400, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=400, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [135]:
model_sentiment.forward(xb,yb,xlen)

(tensor([-0.0855, -0.1155, -0.0758, -0.0851, -0.1091, -0.0973, -0.0949, -0.0326,
         -0.0959, -0.0986, -0.0861, -0.0914, -0.0610, -0.1141, -0.0977, -0.0875,
         -0.0881, -0.1308, -0.0822, -0.0924, -0.0646, -0.1058, -0.0978, -0.0762,
         -0.0642, -0.1176, -0.0543, -0.1169, -0.1110, -0.1244, -0.1252, -0.1187,
         -0.1283, -0.0615, -0.0693, -0.1176, -0.0966, -0.1196, -0.1157, -0.0867,
         -0.0767, -0.0912, -0.1194, -0.0905, -0.0952, -0.0872, -0.0940, -0.0750,
         -0.1190, -0.0672, -0.0660, -0.0937], grad_fn=<ViewBackward>),
 tensor(0.6936, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [136]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [137]:
preds.view(-1)

tensor([-0.0877, -0.0616, -0.0624, -0.1325, -0.1062, -0.1175, -0.1209, -0.0730,
        -0.1155, -0.1202, -0.0677, -0.0789, -0.0753, -0.1177, -0.0672, -0.1035,
        -0.1001, -0.0597, -0.0951, -0.0810, -0.1159, -0.1006, -0.0979, -0.1038,
        -0.0701, -0.0946, -0.0999, -0.0972, -0.0835, -0.1327, -0.0987, -0.1130,
        -0.0799, -0.0899, -0.1037, -0.0677, -0.1096, -0.1281, -0.0845, -0.1002,
        -0.0692, -0.1216, -0.0992, -0.0805, -0.1125, -0.1083, -0.1001, -0.0852,
        -0.0986, -0.1039, -0.0929, -0.0893], grad_fn=<ViewBackward>)

In [138]:
yb

tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        1, 0, 1, 1])

In [139]:
preds.size()

torch.Size([52])

In [140]:
yb.size()

torch.Size([52])

In [141]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.5000)

In [142]:
from sklearn.metrics import roc_auc_score

In [143]:
roc_auc_score(yb,preds)

0.48372781065088755

In [144]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [154]:
# Weight Decay Schedule
tempstart=1e-6
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*7

0 1e-06
1 7e-06
2 4.9e-05
3 0.000343
4 0.002401
5 0.016807


In [146]:
tempstart=5e-3
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*0.5

0 0.005
1 0.0025
2 0.00125
3 0.000625
4 0.0003125
5 0.00015625


In [147]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None,\
                 cycle_mult=0,lr_decay=0.5,wd_mult=9):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
        self.cycle_mult,self.lr_decay=cycle_mult,lr_decay
        self.wd_mult=wd_mult
        for param_group in self.optimizer.param_groups:
            self.start_lr=param_group['lr']
            self.start_wd=param_group['weight_decay']
        self.wd=self.start_wd
        self.lr=self.start_lr
        self.n_epoch=0
        
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

     
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        
        if self.cycle_mult > 0:
            reset_cycle=self.cycle_mult
        
        for epoch in range(n_epochs):                

            self.n_epoch+=1
            loss,acc=self.run_epoch(dltrain,True)
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Learning rate {self.lr} Weight Decay {self.wd} Train Loss:{loss} Train Accuracy:{acc} Valid Loss:{lossv} Valid Accuracy:{accv}')
        
            if self.cycle_mult > 0:
                if self.n_epoch==reset_cycle:
                    self.lr=self.start_lr
                    #self.wd=self.start_wd
                    reset_cycle=epoch+reset_cycle
                else:
                    self.lr*=self.lr_decay  
                    self.wd*=self.wd_mult
                
                
            for param_group in self.optimizer.param_groups:
                param_group['lr']=self.lr
                param_group['weight_decay']=self.wd

In [148]:
device="cuda:0"

In [149]:
n_layers

2

In [150]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [151]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [152]:
len(dltrain),len(dlvalid)

(481, 481)

In [153]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [155]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,1000,0.25,cycle_mult=6,wd_mult=7)

In [156]:
model_sentiment.freeze_embedding()

In [157]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [158]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [159]:
learner.run_epochs(dltrain,dlvalid,6)

Epoch:0 Learning rate 0.005 Weight Decay 1e-07 Train Loss:0.3928912094873599 Train Accuracy:0.8112026559587824 Valid Loss:0.23318554701868313 Valid Accuracy:0.9087078594864034
Epoch:1 Learning rate 0.0025 Weight Decay 7e-07 Train Loss:0.2042975546927685 Train Accuracy:0.919842514451477 Valid Loss:0.19093499751664744 Valid Accuracy:0.9243483508698905
Epoch:2 Learning rate 0.00125 Weight Decay 4.9e-06 Train Loss:0.16278504128732454 Train Accuracy:0.9392452002067327 Valid Loss:0.1877760647906459 Valid Accuracy:0.925987562493822
Epoch:3 Learning rate 0.000625 Weight Decay 3.43e-05 Train Loss:0.13788132545327197 Train Accuracy:0.9484008040101017 Valid Loss:0.18523206577680834 Valid Accuracy:0.9298417140689065
Epoch:4 Learning rate 0.0003125 Weight Decay 0.0002401 Train Loss:0.13460063262070043 Train Accuracy:0.9500160275526701 Valid Loss:0.19091754032225347 Valid Accuracy:0.9252839024002487
Epoch:5 Learning rate 0.00015625 Weight Decay 0.0016807 Train Loss:0.16353188094193127 Train Accuracy

In [160]:
optimizer_sgd=torch.optim.SGD(model_sentiment.parameters(),lr=1e-2,momentum=0.9, weight_decay=wd)

In [161]:
learner=Learner(model_sentiment,optimizer_sgd,accuracy_binomial,device,bptt,1000,0.25,cycle_mult=6,wd_mult=7)

In [162]:
learner.run_epochs(dltrain,dlvalid,3)

Epoch:0 Learning rate 0.01 Weight Decay 1e-07 Train Loss:0.16843031244565446 Train Accuracy:0.9357148951651401 Valid Loss:0.19169426282670296 Valid Accuracy:0.9254598178883352
Epoch:1 Learning rate 0.005 Weight Decay 7e-07 Train Loss:0.16119096826630297 Train Accuracy:0.9394610969043818 Valid Loss:0.19637574405717254 Valid Accuracy:0.9245242666058134
Epoch:2 Learning rate 0.0025 Weight Decay 4.9e-06 Train Loss:0.1559735816901476 Train Accuracy:0.9399888420055413 Valid Loss:0.1932840056004866 Valid Accuracy:0.9254838063414528


In [77]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.928')
torch.save(optimizer.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.928')
torch.save (model_sentiment,f'{PATH}/inter/master_plannery_nounfreeze_0.928')
torch.save (optimizer,f'{PATH}/inter/master_plannery_nounfreeze_0.928')
torch.save (learner,f'{PATH}/inter/master_plannery_nounfreeze_0.928')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [78]:
model_sentiment.unfreeze_embedding()

In [79]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [80]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.09578512791544198  0.9665384948253631
Batch:100 0.11093452228233218  0.9619231098890304
Batch:150 0.11958941758920749  0.9565384960174561
Batch:200 0.12323234149254858  0.9540384972095489
Batch:250 0.12595805583149194  0.953384651184082
Batch:300 0.12950655991211535  0.9516667028268179
Batch:350 0.13006711812423807  0.9508242118358612
Batch:400 0.1332779105519876  0.9496634976565838
Batch:450 0.13643893861522277  0.9479060191578336
Batch:50 0.2287508177757263  0.9142308092117309
Batch:100 0.20826085858047008  0.9211538857221604
Batch:150 0.20082650646567346  0.9230769614378611
Batch:200 0.201063946262002  0.9224038842320442
Batch:250 0.20018781180679798  0.9221538841724396
Batch:300 0.2007621809716026  0.9214743969837824
Batch:350 0.19998458223683493  0.9215384999343327
Batch:400 0.19938704310916364  0.9215865768492222
Batch:450 0.201514434557822  0.9210684147146013
Epoch:0 Learning rate 0.0032 Train Loss:0.13788384909348528 Train Accuracy:0.947013469842764 Valid Loss:0.2019

KeyboardInterrupt: 

In [113]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.1796993599832058  0.9576923358440399
Batch:100 0.17213856488466261  0.9584615689516067
Batch:150 0.16667891293764114  0.9589743900299073
Batch:200 0.16918746769428253  0.9588461855053901
Batch:250 0.1691615637242794  0.9592308008670807
Batch:300 0.169596492399772  0.9590384934345881
Batch:350 0.16886687725782396  0.9589560759067536
Batch:400 0.17080709227360785  0.9581250323355198
Batch:450 0.16847730916407372  0.9587607159879472
Epoch:0 Loss:0.16862219942272824
Batch:50 0.22200749382376672  0.9234615743160248
Batch:100 0.2091979483515024  0.9265385007858277
Batch:150 0.20304773584008218  0.9300000381469726
Batch:200 0.20253973938524722  0.9300961908698082
Batch:250 0.2047864469587803  0.9294615759849548
Batch:300 0.20614869686464468  0.9289743967851003
Batch:350 0.2068827559053898  0.9282967409065791
Batch:400 0.20708336601033805  0.9288461914658547
Batch:450 0.20918306630518702  0.9283761061562432
Epoch:0 Loss:0.16862219942272824 Accuracy:0.9590476893833422 Loss:0.20871845

In [114]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.14831322476267814  0.9669231069087982
Batch:100 0.15890183448791503  0.9651923382282257
Batch:150 0.15571161108712356  0.966153877178828
Batch:200 0.15806500744074584  0.9662500303983689
Batch:250 0.1604635378420353  0.9652307999134063
Batch:300 0.16394074910630782  0.9637820820013682
Batch:350 0.16396197034844331  0.9634615690367563
Batch:400 0.16163033758290113  0.9640384916961193
Batch:450 0.16222475753062301  0.9635043039586809
Epoch:0 Loss:0.16214304859604756
Batch:50 0.22104861378669738  0.923076958656311
Batch:100 0.20808142654597758  0.9263461923599243
Batch:150 0.20183727160096168  0.9297436273097992
Batch:200 0.20117236562073232  0.9301923441886902
Batch:250 0.20324968230724336  0.9299231135845184
Batch:300 0.20449111166099707  0.9294872166713078
Batch:350 0.2054740560054779  0.9289011357511793
Batch:400 0.2057606272958219  0.929423113912344
Batch:450 0.2078460860417949  0.9288034560945299
Epoch:0 Loss:0.16214304859604756 Accuracy:0.963365615033806 Loss:0.207645952

In [115]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.16885913476347925  0.9615384936332703
Batch:100 0.15936354756355287  0.9651923406124115
Batch:150 0.1592694428563118  0.9644872109095256
Batch:200 0.15632881922647357  0.9650000315904618
Batch:250 0.155020054936409  0.9660000314712525
Batch:300 0.1532587735603253  0.9660897747675578
Batch:350 0.15422616588217872  0.9660989323684147
Batch:400 0.15426115840673446  0.9659615701436997
Batch:450 0.1547477538469765  0.9656410574913025
Epoch:0 Loss:0.15475054663797674
Batch:50 0.22095745116472243  0.9238461887836457
Batch:100 0.20784359097480773  0.9267308074235916
Batch:150 0.20166147058208783  0.9303846526145935
Batch:200 0.201059164442122  0.9307692670822143
Batch:250 0.20317641404271125  0.9301538825035095
Batch:300 0.20448637386163077  0.9293590112527211
Batch:350 0.205431331630264  0.9285714653560093
Batch:400 0.20577694775536656  0.9293269602954388
Batch:450 0.2079033212032583  0.9288034560945299
Epoch:0 Loss:0.15475054663797674 Accuracy:0.9655805533740228 Loss:0.20764656368

In [179]:
len(dltrain)

433